# YOLO模型说明

### 模型
<img src="model_data/fig_model2.png" />

### def yolo_body(inputs, num_anchors, num_classes)
    生成 yolo_body模型
    模型输入:
        yolo_model.input --- inputs,shape=>(batch_size,416,416,3)
    模型输出:
        yolo_model.output => [y1,y2,y3]
        y1 --- shape=>(batch_size,13,13,255)=>(batch_size,13,13,3,85)
        y2 --- shape=>(batch_size,26,26,255)=>(batch_size,26,26,3,85)
        y3 --- shape=>(batch_size,52,52,255)=>(batch_size,52,52,3,85)
### anchors分配策略
    anchors分配策略：一共有9个anchor,三个模型输出[y1,y2,y3]
    anchor从小到大排序为0,1,2,3,4,5,6,7,8
    y1的分辨率最小(13x13),其次为y2的分别率为(26x26),y3的分辨率最高(52x52)
    高分辨率更容易检测小物体，因此，小的anchor分配给高分辨率的输出，大的anchor分配给小分辨率的输出
    anchors(0,1,2)=>y3,(0,1,2)再分配给不同的滤波层[0:3]
    anchors(3,4,5)=>y2,同上
    anchors(6,7,8)=>y1,同上
    
    9个anchor分配给[y1,y2,y3]，每个yx得3个anchor，三个滤波层每个按序号得1个anchor
    
### 边框回归公式推导
    Faster-RCNN anchor的预测公式
$t_{x}=(x-x_{a}/w_{a})$   $\qquad$  $t_{y}=(y-y_{a}/h_{a})$

$t_{w}=log(w/w_{a})$      $\qquad$  $t_{h}=log(h/h_{a})$

$t_{x}^*=(x^*-x_{a}/w_{a})$   $\qquad$  $t_{y}^*=(y^*-y_{a}/h_{a})$

$t_{w}^*=log(w^*/w_{a})$      $\qquad$  $t_{h}^*=log(h^*/h_{a})$

公式中，符号的含义解释一下：x 是坐标预测值，$x_{a}$ 是anchor坐标（预设固定值），$x^∗$ 是坐标真实值（标注信息），其他变量 y，w，h 以此类推，t 变量是偏移量。

    预测相对于grid cell的坐标位置的办法
    
<img src="model_data/fig_model3.png">    

$t_{x},t_{y},t_{w},t_{h}$通过sigmoid限制在0-1之间;$p_{w},p_{h}$是anchor的宽度和高度,$c_{x},c_{y}$是grid cell左上角距离

$b_{x},b_{y},b_{w},b_{h}$为预测值

# 模型训练
### 模型训练执行流程线索

    构建模型 create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5')
      构建模型 model = Model([model_body.input, *y_true], model_loss)
          模型输入
              输入数据 image_input = Input(shape=(None, None, 3))
              标注数据 y_true => [y1,y2,y3]=> [(?,13,13,3,25),(?,26,26,3,25),(?,52,52,3,25)]
          模型输出 model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
                    arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
                    [*model_body.output, *y_true])
    编译模型
    训练模型
        数据生成器 data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)
            数据增强/提取boxs (image,box)=get_random_data(annotation_lines[i], input_shape, random=True)
                image --- 图像增强处理，归一化处理
                box   --- 与image同步变形，未归一化处理，shape=>(max_boxs,5)
            标注数据处理 y_true=preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
                    box_data.shape=>(batch_size,max_boxs,5)
                    提取图像有效框，匹配最佳anchor，在对应y_true位置填充box,obj,cls
                    1.遍历批数据的每个图像=>m
                    2.提取图像有效框=>wh
                    3.计算每个框匹配的最佳anchor(IoU计算)=>n=>k
                       根据n找到所属层(y1,y2 or y3)和filter_index
                    4.查找最佳anchor属于那个层(anchor_mask)=>l
                    5.换算外框中心所属grid cell=>(j,i)
                    6.提取类别ID=>c
                    7.填充 y_true
                        y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]  #x,y,w,h
                        y_true[l][b, j, i, k, 4] = 1                       #所有检测类别统一视为Object
                        y_true[l][b, j, i, k, 5+c] = 1                     #对应类别序号上设置标记
                        x,y,w,h是归一化值，相对值0-1之间
        损失函数 yolo_loss(args, anchors, num_classes, ignore_thresh=.5, print_loss=False)
            

# 模型测试

### 模型测试执行流程线索
    设置执行参数 => params
    创建YOLO对象=> yolo=YOLO(**params)
     |- 读取分类列表 => self.class_names=self._get_classes()
     |- 读取锚框数组 => self.anchors=self._get_anchors()
     |- 构造测试模型 => self.boxes, self.scores, self.classes = self.generate()
     |   |- 构造YOLO模型 => self.yolo_model=yolo_body(Input(shape=(None,None,3)),                                              
     |   |   |                                       num_anchors//3, num_classes)
     |   |   |-构造darknet => darknet=Model(inputs, darknet_body(inputs))
     |   |   |  |-  x = DarknetConv2D_BN_Leaky(32, (3,3))(x)
     |   |   |  |       return compose(
     |   |   |  |           DarknetConv2D(*args, **no_bias_kwargs),
     |   |   |  |           BatchNormalization(), 
     |   |   |  |           LeakyReLU(alpha=0.1))                    
     |   |   |  |-  x = resblock_body(x, 64, 1)
     |   |   |  |       x = ZeroPadding2D(((1,0),(1,0)))(x)
     |   |   |  |       x = DarknetConv2D_BN_Leaky(num_filters, (3,3), strides=(2,2))(x)
     |   |   |  |       for i in range(num_blocks):
     |   |   |  |           y = compose(
     |   |   |  |                   DarknetConv2D_BN_Leaky(num_filters//2, (1,1)),       #降低1半过滤器
     |   |   |  |                   DarknetConv2D_BN_Leaky(num_filters, (3,3)))(x)       #恢复过滤器数
     |   |   |  |           x = Add()([x,y])                                             #残差
     |   |   |  |       #x.shape=>(batch_size,height/2,width/2,num_filters)
     |   |   |  |       #y.shape=>(batch_size,height/2,width/2,num_filters)
     |   |   |  |       return x                    
     |   |   |  |-  x = resblock_body(x, 128, 2)
     |   |   |  |-  x = resblock_body(x, 256, 8)
     |   |   |  |-  x = resblock_body(x, 512, 8)
     |   |   |  |-  x = resblock_body(x, 1024, 4)
     |   |   |-生成输出y1 => x, y1 = make_last_layers(darknet.output, 512, num_anchors*(num_classes+5))
     |   |   |-生成输出y2 => x, y2 = make_last_layers(darknet.output, 256, num_anchors*(num_classes+5))
     |   |   |-生成输出y2 => x, y2 = make_last_layers(darknet.output, 128, num_anchors*(num_classes+5))
     |   |   |-输出YOLO模型 => return Model(inputs, [y1,y2,y3])
     |   |- 加载参数 => self.yolo_model.load_weights(self.model_path)
     |   |- yolo估计器 => boxes, scores, classes = yolo_eval(
     |   |   |                  self.yolo_model.output       #模型输出:[y1,y2,y3]
     |   |   |                  , self.anchors               #锚点数组:[9 x 2]
     |   |   |                  , len(self.class_names)      #分类数目：80
     |   |   |                  , self.input_image_shape     #原始图像大小:张量,placeholder,模型输入
     |   |   |                  , score_threshold=self.score #得分阈值
     |   |   |                  , iou_threshold=self.iou     #交并比阈值
     |   |   |                  )
     |   |   |- 提取预测结果 => _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
     |   |   |   |             anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
     |   |   |   |- 提取预测边框 => box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
     |   |   |   |                                                  anchors, num_classes, input_shape)
     |   |   |   |- 修正预测边框为真实值 => boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, 
     |   |   |                                                      image_shape)                                
     |   |   |                          boxes=> (ymin,xmin,ymax,xmax)=>(top,left,bottom,right)
     |   |   |- 条件条件过滤：得分 && 非极大值抑制
     |   |- 返回yolo估计器 => return boxes,scores,classes
    读取图像 => image=Image.open(img_file)    
    对象检测 => r_image=yolo.detect_image(image)
     |- 图像数据预处理：修正为标准尺寸(self.model_image_size)，归一化处理，扩展维度
     |- 执行评估模型(yolo_eval)=> out_boxes, out_scores, out_classes = self.sess.run(
     |          [self.boxes, self.scores, self.classes],
     |          feed_dict={
     |              self.yolo_model.input: image_data,
     |              self.input_image_shape: [image.size[1], image.size[0]],
     |              K.learning_phase(): 0
     |          })
     |- 返回结果图 => 原始图与检测框的合成图
    显示结果 => plt.imshow(r_image)
##### def yolo_eval(yolo_outputs, anchors,  num_classes, image_shape, max_boxes=40, score_threshold=.6,iou_threshold=.5)
    模型测试
    模型输入:
        yolo_model.input --- inputs,shape=>(batch_size,416,416,3)
        image_shape -------- 图像真实尺寸，如(533,400)
    模型输出:boxes, scores, classes
        boxes --- 预测外框,[n x 4],=>()
        scores--- 预测分数,[n x 1]
        classes-- 预测类别,[n x 1]